In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
results = model("bus.jpg")

In [ ]:
%matplotlib inline

import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

def visualize_objects(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run inference
    results = model(image)

    # Convert image to PIL format if it's numpy array
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    draw = ImageDraw.Draw(image)

    # Try to load a nice font, fall back to default if not available
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 20)
    except:
        font = ImageFont.load_default()

    # Generate random colors for each class
    np.random.seed(42)
    colors = np.random.randint(0, 255, size=(len(results[0].names), 3)).tolist()

    # Get the boxes and confidence scores
    boxes = results[0].boxes

    for box in boxes:
        # Get box coordinates
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())

        # Skip if confidence is below threshold
        if conf < 0.3:
            continue

        # Draw bounding box
        color = tuple(colors[cls])
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)

        # Add label with class name and confidence
        label = f"{results[0].names[cls]}: {conf:.2f}"
        label_size = draw.textsize(label, font=font) if hasattr(draw, 'textsize') else (
            draw.textlength(label, font=font), font.size)

        # Draw label background
        draw.rectangle([x1, y1 - label_size[1], x1 + label_size[0], y1],
                    fill=color)

        # Draw label text
        draw.text((x1, y1 - label_size[1]), label, fill=(255, 255, 255),
                font=font)

    output_image = image

    # Create figure with specified size
    plt.figure(figsize=(15, 10))

    # Display the image
    # Convert PIL image back to numpy array if needed
    if isinstance(output_image, Image.Image):
        output_image = np.array(output_image)

    plt.imshow(output_image)
    plt.axis('off')
    return results

In [ ]:
results = visualize_objects("bus.jpg")

# Print detection summary
print("\nDetection Summary:")
print("-----------------")
for box in results[0].boxes:
    conf = float(box.conf[0].cpu().numpy())
    cls = int(box.cls[0].cpu().numpy())
    if conf >= 0.3:
        print(f"Class: {results[0].names[cls]}, Confidence: {conf:.2f}")
